![Cartoon of telecom customers](IMG_8811.png)


The telecommunications (telecom) sector in India is rapidly changing, with more and more telecom businesses being created and many customers deciding to switch between providers. "Churn" refers to the process where customers or subscribers stop using a company's services or products. Understanding the factors that influence keeping a customer as a client in predicting churn is crucial for telecom companies to enhance their service quality and customer satisfaction. As the data scientist on this project, you aim to explore the intricate dynamics of customer behavior and demographics in the Indian telecom sector in predicting customer churn, utilizing two comprehensive datasets from four major telecom partners: Airtel, Reliance Jio, Vodafone, and BSNL:

- `telecom_demographics.csv` contains information related to Indian customer demographics:

| Variable             | Description                                      |
|----------------------|--------------------------------------------------|
| `customer_id `         | Unique identifier for each customer.             |
| `telecom_partner `     | The telecom partner associated with the customer.|
| `gender `              | The gender of the customer.                      |
| `age `                 | The age of the customer.                         |
| `state`                | The Indian state in which the customer is located.|
| `city`                 | The city in which the customer is located.       |
| `pincode`              | The pincode of the customer's location.          |
| `registration_event` | When the customer registered with the telecom partner.|
| `num_dependents`      | The number of dependents (e.g., children) the customer has.|
| `estimated_salary`     | The customer's estimated salary.                 |

- `telecom_usage` contains information about the usage patterns of Indian customers:

| Variable   | Description                                                  |
|------------|--------------------------------------------------------------|
| `customer_id` | Unique identifier for each customer.                         |
| `calls_made` | The number of calls made by the customer.                    |
| `sms_sent`   | The number of SMS messages sent by the customer.             |
| `data_used`  | The amount of data used by the customer.                     |
| `churn`    | Binary variable indicating whether the customer has churned or not (1 = churned, 0 = not churned).|


# Project Instructions

Does Logistic Regression or Random Forest produce a higher accuracy score in predicting telecom churn in India?

- Load the two CSV files into separate DataFrames. Merge them into a DataFrame named churn_df. Calculate and print churn rate, and identify the categorical variables in churn_df.
- Convert categorical features in churn_df into features_scaled. Perform feature scaling separating the appropriate features and scale them. Define your scaled features and target variable for the churn prediction model.
- Split the processed data into training and testing sets giving names of X_train, X_test, y_train, and y_test using an 80-20 split, setting a random state of 42 for reproducibility.
- Train Logistic Regression and Random Forest Classifier models, setting a random seed of 42. Store model predictions in logreg_pred and rf_pred.
- Assess the models on test data. Assign the model's name with higher accuracy ("LogisticRegression" or "RandomForest") to higher_accuracy.

## Resources 
Check resources that can help you solve the problem.

### LESSONS

[Inner join](https://campus.datacamp.com/courses/joining-data-with-pandas/data-merging-basics?ex=1)
[Random Forests (RF)](https://campus.datacamp.com/courses/machine-learning-with-tree-based-models-in-python/bagging-and-random-forests?ex=7)
[Logistic regression](https://campus.datacamp.com/courses/supervised-learning-with-scikit-learn/fine-tuning-your-model-3?ex=4)

### SLIDES

[Classification report in scikit-learn](https://campus.datacamp.com/pdf/web/viewer.html?file=https://projector-video-pdf-converter.datacamp.com/28314/chapter3.pdf#page=18)

### CHEATSHEETS

[Scikit-Learn Cheat Sheet: Python Machine Learning](Scikit-Learn_Cheat_Sheet.pdf)


In [470]:
# Import libraries and methods/functions
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [471]:
# Load the two CSV files into separate DataFrames.
demographics_df = pd.read_csv("telecom_demographics.csv")
demographics_df.head()  

,customer_id,telecom_partner,gender,age,state,city,pincode,registration_event,num_dependents,estimated_salary
0,15169,Airtel,F,26,Himachal Pradesh,Delhi,667173,2020-03-16,4,85979
1,149207,Airtel,F,74,Uttarakhand,Hyderabad,313997,2022-01-16,0,69445
2,148119,Airtel,F,54,Jharkhand,Chennai,549925,2022-01-11,2,75949
3,187288,Reliance Jio,M,29,Bihar,Hyderabad,230636,2022-07-26,3,34272
4,14016,Vodafone,M,45,Nagaland,Bangalore,188036,2020-03-11,4,34157


In [472]:
usage_df = pd.read_csv("telecom_usage.csv")
usage_df.head()

,customer_id,calls_made,sms_sent,data_used,churn
0,15169,75,21,4532,1
1,149207,35,38,723,1
2,148119,70,47,4688,1
3,187288,95,32,10241,1
4,14016,66,23,5246,1


In [473]:
# Merge them into a DataFrame named churn_df.
churn_df = demographics_df.merge(usage_df, on="customer_id")
churn_df.head()

,customer_id,telecom_partner,gender,age,state,city,pincode,registration_event,num_dependents,estimated_salary,calls_made,sms_sent,data_used,churn
0,15169,Airtel,F,26,Himachal Pradesh,Delhi,667173,2020-03-16,4,85979,75,21,4532,1
1,149207,Airtel,F,74,Uttarakhand,Hyderabad,313997,2022-01-16,0,69445,35,38,723,1
2,148119,Airtel,F,54,Jharkhand,Chennai,549925,2022-01-11,2,75949,70,47,4688,1
3,187288,Reliance Jio,M,29,Bihar,Hyderabad,230636,2022-07-26,3,34272,95,32,10241,1
4,14016,Vodafone,M,45,Nagaland,Bangalore,188036,2020-03-11,4,34157,66,23,5246,1


In [474]:
# Calculate and print churn rate
print("Shape:\n", churn_df.shape)
print("Total Observations: ", total_samples := churn_df["churn"].count())
print("Total Churn: ", total_churn := churn_df["churn"].sum())
print(churn_rate := churn_df['churn'].value_counts() / len(churn_df))


Shape:
 (6500, 14)
Total Observations:  6500
Total Churn:  1303
churn
0    0.799538
1    0.200462
Name: count, dtype: float64


In [475]:
# Identify categorical variables
print(churn_df.info())
categorical_variables = [
    "telecom_partner",
    "gender",
    "state",
    "city",
    "registration_event", # used in model answer - I find this one a bit weird
]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6500 entries, 0 to 6499
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         6500 non-null   int64 
 1   telecom_partner     6500 non-null   object
 2   gender              6500 non-null   object
 3   age                 6500 non-null   int64 
 4   state               6500 non-null   object
 5   city                6500 non-null   object
 6   pincode             6500 non-null   int64 
 7   registration_event  6500 non-null   object
 8   num_dependents      6500 non-null   int64 
 9   estimated_salary    6500 non-null   int64 
 10  calls_made          6500 non-null   int64 
 11  sms_sent            6500 non-null   int64 
 12  data_used           6500 non-null   int64 
 13  churn               6500 non-null   int64 
dtypes: int64(9), object(5)
memory usage: 711.1+ KB
None


In [476]:
# Extract label / target
target = churn_df['churn']

# Drop unwanted columns
features = churn_df.drop(["customer_id", "churn"],axis='columns')

In [477]:
# Create an instance of OneHotEncoder
encoder = OneHotEncoder(handle_unknown="ignore")

# Fit the encoder to the categorical columns
encoded = encoder.fit_transform(features[categorical_variables])

# Convert the encoded array to a dataframe
encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out())

# Join the encoded dataframe with the original dataframe
features_ohe = features.join(encoded_df)

# Drop the original categorical columns
features_ohe.drop(categorical_variables, axis=1, inplace=True)

# Model answer uses this: 
#features_scaled = pd.get_dummies(features, columns=['telecom_partner', 'gender', 'state', 'city', 'registration_event'])


In [478]:
# Splitting the dataset into the Training set and Test set before scaling - not mentioned in the project but is best practice
X = features_ohe.values
y = target.values

In [479]:
# Splitting the dataset into the Training set and Test set before scaling - not mentioned in the project but is best practice
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [480]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [481]:
# Train Logistic Regression model
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train) 

logreg_pred = logreg.predict(X_test) 

In [482]:
# Random Forest Classifier model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train) 

rf_pred = rf.predict(X_test) 

In [483]:
# Assess the models on test data. First a classification_report
print("Logistic Regression: \n", classification_report(y_test, logreg_pred))
print('')
print("Random Forrest: \n", classification_report(y_test, rf_pred))



Logistic Regression: 
               precision    recall  f1-score   support

           0       0.79      0.90      0.84      1027
           1       0.21      0.10      0.14       273

    accuracy                           0.73      1300
   macro avg       0.50      0.50      0.49      1300
weighted avg       0.67      0.73      0.69      1300


Random Forrest: 
               precision    recall  f1-score   support

           0       0.79      1.00      0.88      1027
           1       0.00      0.00      0.00       273

    accuracy                           0.79      1300
   macro avg       0.39      0.50      0.44      1300
weighted avg       0.62      0.79      0.70      1300



In [484]:
# And now the confusion_matrix
print("Logistic Regression: \n", confusion_matrix(y_test, logreg_pred))
print('')
print("Random Forrest: \n", confusion_matrix(y_test, rf_pred))


Logistic Regression: 
 [[920 107]
 [245  28]]

Random Forrest: 
 [[1026    1]
 [ 273    0]]


In [485]:
higher_accuracy = "RandomForest"